In [1]:
# !pip install --upgrade openai

In [1]:
import pandas as pd
import numpy as np
import random
import configparser
import shelve
import re

import openai

In [3]:
# %%
config = configparser.ConfigParser(interpolation=None)
config.read("config_openai.ini")
 
openai.organization = config["openai"]["org_id"]
openai.api_key = config["openai"]["secret_key"]

In [4]:
df = pd.read_csv('sample_bios_after109nd.csv')
df_out = df[['id', 'bio']].copy()

In [5]:
model = 'gpt-3.5-turbo'
temperature = 0

In [6]:
prompt_pre = open("prompt.txt", "r").read()

In [10]:
with shelve.open("./db_hold/api.db") as api_db:
    init_key_list = list(api_db.keys())
    for row in df.itertuples():
        print(row.Index)
        cur_messages = [
            {
                "role": "user",
                "content": prompt_pre + row.bio,
            },
        ]
        cur_key = row.id
        if cur_key in init_key_list:
            api_result = api_db[cur_key]
        else:
            success = False
            while not success:
                # t0 = time.time()
                try:
                    api_result = openai.ChatCompletion.create(
                        model=model,
                        messages=cur_messages,
                        temperature=temperature,
                    )
                    api_db[cur_key] = api_result
                    print(api_result["usage"]["total_tokens"])
                    success = True
                except openai.error.RateLimitError:
                    print("Rate limited")
                except openai.error.APIError:
                    print("Gateway Timeout")
                # if time.time() - t0 < 16:
                #     time.sleep(16 - (time.time() - t0))
        cur_out = api_result["choices"][0]["message"]["content"]
        cur_finish = api_result["choices"][0]["finish_reason"]
        if cur_finish != "stop":
            print("WARNING: did not finish")
        df_out.loc[row.Index, "api_out"] = cur_out

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
df_out.to_csv('gpt_bio_result.csv')